# Generate structured cases for the following scenarios

1. 52 year old man comes to ED complaining of chest pain
2. 45 year old man comes to outpatient clinic complaining of bloody urine
3. 49 year old man comes to clinic complaining of back pain 
4. 28 year old man walks into ED after a motor vehicle accident
5. 32 year old woman walks into clinic with a positive pregnancy test
6. 11 year old girl comes to clinic with an elevated glucose test from school nurse
7. 69 year old man complains of left arm pain
8. 63 year old woman comes back to clinic for follow-up of diabetes
9. 28 year old lady comes to ED after sexual assault
10. 40 year old woman complains of right calf pain
11. 65 year old man presents with new onset hoarseness of voice
12. 70 year old lady presenting to clinic with neck pain
13. 55 year old female admitted to ED for abdominal pain
14. 38 year old woman presents to clinic with headache
15. 35 year old lady complains of irregular menses
16. 24 year old young female presents to clinic with regular pain during sex
17. 80 year old elderly gentleman complaining of hearing loss
18. 5 year old girl coming in to clinic with jaundice
19. 8 month old boy presents to ED with fever
20. 22 year old young gentleman presents to clinic with cough
21. 49 year old lady presenting to clinic with jaundice
22. 55 year old male complains of dizziness
23. 35 year old young lady presenting to the clinic with knee pain
24. 28 year old young man complains of heel pain
25. 19 month old girl presenting to clinic with fever
26. 58 year old woman complains of chronic cough
27. 59 year old man complains of chronic fatigue
28. 61 year old man comes to clinic for followup on hypertension
29. 23 year old young lady complains of sleeping issues
30. 2 year old girl presents to clinic with persistent noisy breathing
31. 24 year old woman presents to ED with abdominal pain
32. 68 year old elderly lady complaining of forgetfulness and disorientation
33. 49 year old man came to clinic complaining of fatigue
34. 32 year old young lady presents to clinic with fatigue
35. 26 year old man complains of visual and auditory hallucinations
36. 29 year old woman presents in clinic for pre employment physical
37. 59 year old man presents to ED with bloody stools
38. 61 year old man presents to clinic with tremors and walking problems
39. 32 year old young woman came to clinic with weight gain
40. 5 month old girl presents to ED with multiple episodes of diarrhoea
41. 7 year old boy presents to clinic with bed wetting
42. 13 month old girl presents to ED with seizures
43. 25 year old man complains of sore throat
44. 54 year old man presents to ED with fall and loss of consciousness

In [442]:
import pandas as pd

details = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Details")
history = pd.read_excel("Data/BaseCase.xlsx", sheet_name="History Taking", keep_default_na=False)
physical = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Physical Examination")
diagnosis = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Differential Diagnosis")
Investigations = pd.read_excel("Data/BaseCase.xlsx", sheet_name="Investigations")

In [444]:
from importlib import reload
import simulator
reload(simulator.generators)

history_taking_example = "\n".join(
  [f"Question: {row['Question']} | Response: {row['Patient Response']}"
    for index, row in history.iterrows()])

case_generator = simulator.generators.CaseGenerator(
  model_name="gpt-4o",
  case_stem = "65 year old man presents with new onset hoarseness of voice",
  basic_details_example= details["Case Prompt"].values[0],
  vitals_example = details["Vitals"].values[0],
  physical_presentation_example = details["Physical presentation"].values[0],
  challenging_question_example = details["Challenging Questions to Ask"].values[0],
  history_taking_example = history_taking_example
)

In [445]:
output = case_generator.generate_basic_details()

In [446]:
output

['Edward Green, a 65-year-old male, presents to the clinic with a new onset of hoarseness of voice.',
 'BP: 130/80 mm Hg\nTemp: 98.4°F (36.9°C)\nRR: 18/minute\nHR: 72/minute, regular',
 None,
 '“Could this be something serious like cancer?”']

In [447]:
output = case_generator.generate_history()

In [448]:
for item in output.ChecklistItems:
  print(item.question)
  print(item.response)
  print()

Chief complaint
Hoarseness of voice

Onset
Started about a week ago

Precipitating events
Noticed it after a long day of talking

Progression
It has been getting worse

Alleviating / Exacerbating factors
Resting my voice helps a bit

Difficulty swallowing
Slight difficulty with large or dry foods

Throat clearing
I occasionally need to clear my throat

Associated symptoms
No pain, but my throat feels a bit scratchy

Previous episodes
Never had this before

Current medications
None

Past medical history
I have high blood pressure, managed with diet

Past surgical history
Appendectomy 20 years ago

Family history
My father had a stroke at age 70. My mother is healthy.

Occupation
Retired teacher

Alcohol use
I drink socially, maybe once a week

Tobacco history
I smoked for 30 years, quit 5 years ago

Duration of tobacco use
30 years

Exercise
I walk daily for about 30 minutes

Diet
I try to eat a balanced diet

Drug Allergies
None



In [449]:
output = case_generator.generate_physical_exam()

In [450]:
for item in output.ChecklistItems:
  print(item.technique)
  print(item.justification)
  print()

Inspection of the throat and oral cavity
To visually assess for any lesions, swelling, or abnormalities that could be contributing to the hoarseness.

Palpation of the neck
To check for any lymphadenopathy or masses that could indicate an underlying pathology.

Auscultation of the lungs
To rule out any respiratory issues that might be associated with the hoarseness.

Examination of the vocal cords (indirect laryngoscopy)
To directly visualize the vocal cords and assess for any abnormalities such as nodules, polyps, or paralysis.

Neurological examination
To assess for any neurological deficits that could be contributing to the hoarseness, especially given the patient's history of smoking.

Examination of the cranial nerves
To ensure there is no cranial nerve involvement, particularly the vagus nerve, which could affect voice.

Assessment of swallowing function
To evaluate the slight difficulty with swallowing and ensure there is no significant dysphagia.

Examination of the thyroid gla

In [451]:
output = case_generator.generate_investigations()

In [452]:
for item in output.ChecklistItems:
  print(item.investigation)
  print(item.justification)
  print()

Laryngoscopy
To directly visualize the vocal cords and larynx to identify any structural abnormalities, lesions, or signs of inflammation that could explain the hoarseness.

Neck CT or MRI
To assess for any masses, tumors, or structural abnormalities in the neck region that might be affecting the vocal cords or surrounding structures.

Thyroid function tests
To rule out thyroid dysfunction, as thyroid issues can sometimes present with hoarseness.

Chest X-ray
To check for any mediastinal masses or lung pathology that might be impinging on the recurrent laryngeal nerve, leading to hoarseness.

Complete blood count (CBC)
To check for signs of infection or inflammation that might be contributing to the symptoms.

ESR or CRP
To assess for any underlying inflammatory processes that could be causing the hoarseness.



In [453]:
output = case_generator.generate_ddx()

In [454]:
for item in output.ChecklistItems:
  print(item.diagnosis)
  print(item.justification)
  print()

Laryngeal cancer
The patient is a 65-year-old male with a history of smoking for 30 years, presenting with new-onset hoarseness of voice, which is a common symptom of laryngeal cancer. The progressive nature of the hoarseness and the slight difficulty swallowing large or dry foods further support this possibility.

Vocal cord nodules or polyps
The patient has a history of prolonged voice use (retired teacher) and noticed hoarseness after a long day of talking. Vocal cord nodules or polyps are common in individuals who use their voice extensively and can cause hoarseness.

Gastroesophageal reflux disease (GERD)
The patient reports occasional throat clearing and a scratchy throat, which can be associated with GERD. GERD can lead to laryngopharyngeal reflux, causing hoarseness and throat irritation.

Vocal cord paralysis
The patient presents with hoarseness and slight difficulty swallowing, which can be symptoms of vocal cord paralysis. This condition can occur due to nerve damage or othe

In [455]:
case_generator.export_case(file_name="11")

>>> Exporting basic details
>>> Exporting history
>>> Exporting Physical Exam
>>> Exporting Investigations
>>> Exporting Differential Diagnosis
